In [3]:
import yfinance as yf
import keras.backend as K
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler ,StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report,accuracy_score, mean_squared_error,mean_absolute_percentage_error,r2_score
import tensorflow as tf
from tensorflow import keras
from keras.layers import LSTM,Dropout, BatchNormalization, Dense
from keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau, CSVLogger
from keras.models import Sequential
from keras.optimizer_v2.adam import Adam
from keras.optimizer_v2.rmsprop import RMSprop
import talib

In [4]:
data = yf.download('0005.hk',"2017-09-28","2021-09-24")
data.head()

[*********************100%***********************]  1 of 1 completed


,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,
2017-09-28,76.400002,77.099998,76.050003,76.199997,64.386978,22520013
2017-09-29,76.500000,76.900002,76.300003,76.550003,64.682739,16766252
2017-10-03,76.800003,77.449997,76.699997,77.449997,65.443207,24369212
2017-10-04,77.500000,77.949997,77.500000,77.849998,65.781197,21197563
2017-10-06,77.599998,77.800003,77.199997,77.349998,65.358704,23715110


In [5]:
data['Prediction']=data['Adj Close'].shift(-1)
data.dropna(inplace=True)
data.isnull().sum()

Open          0
High          0
Low           0
Close         0
Adj Close     0
Volume        0
Prediction    0
dtype: int64

In [6]:
data.head()

,Open,High,Low,Close,Adj Close,Volume,Prediction
Date,,,,,,,
2017-09-28,76.400002,77.099998,76.050003,76.199997,64.386978,22520013,64.682739
2017-09-29,76.500000,76.900002,76.300003,76.550003,64.682739,16766252,65.443207
2017-10-03,76.800003,77.449997,76.699997,77.449997,65.443207,24369212,65.781197
2017-10-04,77.500000,77.949997,77.500000,77.849998,65.781197,21197563,65.358704
2017-10-06,77.599998,77.800003,77.199997,77.349998,65.358704,23715110,65.485474


In [7]:
# Build the LSTM model
adam = Adam(lr=0.001)
X=data.drop('Prediction',1).values
X = X.reshape(X.shape[0], 1, X.shape[1])
y=data['Prediction'].values.reshape(-1,1)
X_train, X_test, y_train, y_test = train_test_split(X,y, test_size = 0.2, random_state = 0)
model = Sequential()
model.add(LSTM(60,activation='relu',input_shape=(1, X.shape[2])))
model.add(Dropout(0.8))

# fully connected layers

model.add(Dense(units=50))
model.add( Dropout(rate=0.8))
model.add(Dense(units=25))
model.add(Dropout(0.8))
model.add(Dense(units=10))
model.add(Dropout(rate=0.8))
# output = Dense(units=1)
model.add(Dense(1))
model.compile(
  loss="mean_squared_error",
  optimizer=adam
)

Metal device set to: Apple M1 Pro


/opt/homebrew/Caskroom/miniforge/base/envs/FYP/lib/python3.9/site-packages/keras/optimizer_v2/optimizer_v2.py:355: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  warnings.warn(
/var/folders/8h/w0jn89bj2w5g85gj4ck7w0t00000gn/T/ipykernel_10266/204336600.py:3: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  X=data.drop('Prediction',1).values
2021-12-26 20:43:53.173643: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:305] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2021-12-26 20:43:53.174138: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:271] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


In [8]:
model.summary() 

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm (LSTM)                  (None, 60)                16080     
_________________________________________________________________
dropout (Dropout)            (None, 60)                0         
_________________________________________________________________
dense (Dense)                (None, 50)                3050      
_________________________________________________________________
dropout_1 (Dropout)          (None, 50)                0         
_________________________________________________________________
dense_1 (Dense)              (None, 25)                1275      
_________________________________________________________________
dropout_2 (Dropout)          (None, 25)                0         
_________________________________________________________________
dense_2 (Dense)              (None, 10)                2

In [ ]:
#Fit model with history to check for overfitting
# es = EarlyStopping(monitor='val_loss', mode='min', verbose=1,patience=40, min_delta=0.0001)
history = model.fit(X_train,y_train,epochs=2000,batch_size=20,validation_data=(X_test,y_test),shuffle=False)

2021-12-26 20:43:53.339549: W tensorflow/core/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz


Epoch 1/2000


2021-12-26 20:43:53.673910: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:112] Plugin optimizer for device_type GPU is enabled.


In [ ]:
y_pred = model.predict(X_test)
y_pred

In [ ]:
plt.plot(history.history['loss'], label='train')
plt.plot(history.history['val_loss'], label='test')
plt.legend()

In [ ]:
plt.plot(np.arange(0, len(y_train)), y_train, 'g', label="history")
plt.plot(np.arange(len(y_train), len(y_train) + len(y_test)), y_test, marker='.', label="true")
plt.plot(np.arange(len(y_train), len(y_train) + len(y_test)), y_pred, 'r', label="prediction")
plt.ylabel('Value')
plt.xlabel('Time Step')
plt.legend()
plt.show();

In [ ]:
plt.plot(y_test, marker='.', label="true")
plt.plot(y_pred, 'r',marker='*', label="prediction")
plt.ylabel('Value')
plt.xlabel('Time Step')
plt.legend()
plt.show();

In [ ]:
from sklearn.metrics import r2_score
r2_score(y_test,y_pred)